In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
mnist_tmaxs = [2**exponent for exponent in range(1, 7)]
mnist_error = 100 - np.array([49.21000003814697, 97.74999618530273,98.69999885559082,99.14999604225159,99.2199957370758,99.23999905586243])

cifar_tmaxs = [2**exponent for exponent in range(3, 7)]
cifar_error = 100 - np.array([83.30999612808228, 91.3599967956543, 92.14999675750732, 92.25999712944031])

imagenet_tmaxs = [2**exponent for exponent in range(4, 7)]
imagenet_error = 100 - np.array([54.44, 64.44, 67.24])
imagenet_times = [146, 290, 578]

In [ ]:
plt.scatter(mnist_tmaxs, mnist_error)
plt.plot(mnist_tmaxs, mnist_error, label='MNIST')
plt.scatter(cifar_tmaxs, cifar_error)
plt.plot(cifar_tmaxs, cifar_error, label="CIFAR10")
plt.scatter(imagenet_tmaxs, imagenet_error)
plt.plot(imagenet_tmaxs, imagenet_error, label="ImageNet")
plt.xscale('log')
plt.xlim(right=100)
plt.ylabel("Accuracy error [%]")
plt.xlabel(r"$T_{\mathrm{max}}$ per layer")
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig("simulation_accs.png")